# Task 3
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [2]:
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import os
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from sklearn.model_selection import KFold
from torch.optim import Adam
# Add any other imports you need here
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# The device is automatically set to GPU if available, otherwise CPU
# If you want to force the device to CPU, you can change the line to
# device = torch.device("cpu")
# When using the GPU, it is important that your model and all data are on the
# same device.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Global Variables
global_batch_size = 128
global_num_workers = 16
global_model = models.resnet18(weights='IMAGENET1K_V1')
global_embedding_size = 512

# Debug
verbose = False
testing = False

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 126MB/s]


In [5]:
"""
Transform, resize and normalize the images and then use a pretrained model to extract
the embeddings.
"""
# TODO: define a transform to pre-process the images
# The required pre-processing depends on the pre-trained model you choose
# below.
# See https://pytorch.org/vision/stable/models.html#using-the-pre-trained-models
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to (224, 224)
    transforms.CenterCrop(224),
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/", transform=train_transforms)
# Hint: adjust batch_size and num_workers to your PC configuration, so that you don't
# run out of memory (VRAM if on GPU, RAM if on CPU)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=64,
                          shuffle=False,
                          pin_memory=True, num_workers=16)

# TODO: define a model for extraction of the embeddings (Hint: load a pretrained model,
# more info here: https://pytorch.org/vision/stable/models.html)
model = global_model
model.to(device)
embedding_size = global_embedding_size  # Dummy variable, replace with the actual embedding size once you pick your model
num_images = len(train_dataset)
embeddings = np.zeros((num_images, embedding_size))

# pick your model
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

# TODO: Use the model to extract the embeddings. Hint: remove the last layers of the
# model to access the embeddings the model generates.
# Loop through the dataset and extract embeddings
with torch.no_grad():
  start_idx = 0
  for inputs, _ in train_loader:
    inputs = inputs.to(device)
    outputs = feature_extractor(inputs)
    print(outputs.shape)
    embed = outputs.resize_(outputs.shape[0], outputs.shape[1])
    print(embed.shape)
    batch_size = outputs.shape[0]
    embeddings[start_idx:start_idx+batch_size] = outputs
    start_idx += batch_size
    print(start_idx)
print(embeddings.shape)
print(embeddings)
np.save('/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/embeddings.npy', embeddings)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
64
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
128
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
192
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
256
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
320
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
384
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
448
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
512
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
576
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
640
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
704
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
768
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
832
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
896
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
960
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
1024
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
1088
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])
1152
torch.Size([64, 512, 1, 1]

In [ ]:
with torch.no_grad():
        start_idx = 0
        for inputs, _ in train_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            batch_size = outputs.shape[0]
            embeddings[start_idx:start_idx+batch_size] = outputs.cpu().numpy()
            start_idx += batch_size
np.save('/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/embeddings_2.npy', embeddings)

In [6]:
def get_data(file, train=True):
    """
    Load the triplets from the file and generate the features and labels.

    input: file: string, the path to the file containing the triplets
          train: boolean, whether the data is for training or testing

    output: X: numpy array, the features
            y: numpy array, the labels
    """
    triplets = []
    with open(file) as f:
        for line in f:
            triplets.append(line)

    # generate training data from triplets
    train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/",
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '') for s in train_dataset.samples]
    embeddings = np.load('/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/embeddings.npy')
    # TODO: Normalize the embeddings
    embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)

    #file_to_embedding = {filename: emb for filename, emb in zip(filenames, embeddings)}
    file_to_embedding = {}
    for i in range(len(filenames)):
        file_to_embedding[filenames[i]] = embeddings[i]
    X = []
    y = []
    for t in triplets:
        emb = [file_to_embedding[a] for a in t.split()]
        X.append(np.hstack([emb[0], emb[1], emb[2]]))
        y.append(1)
        if train:
            X.append(np.hstack([emb[0], emb[2], emb[1]]))
            y.append(0)
    X = np.vstack(X)
    y = np.hstack(y)
    return X, y

Hint: adjust batch_size and num_workers to your PC configuration, so that you don't run out of memory (VRAM if on GPU, RAM if on CPU)

In [27]:
def create_loader_from_np(X, y=None, train=True, batch_size=global_batch_size, shuffle=True, num_workers=global_num_workers):
    if train:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float),
                                torch.from_numpy(y).type(torch.long))
    else:
        if y is not None:
            dataset = TensorDataset(torch.from_numpy(X).type(torch.float),
                                    torch.from_numpy(y).type(torch.long))
        else:
            dataset = TensorDataset(torch.from_numpy(X).type(torch.float))
    loader = DataLoader(dataset=dataset,
                        batch_size=batch_size,
                        shuffle=shuffle,
                        pin_memory=True, num_workers=num_workers)
    return loader


TODO: define a model. Here, the basic structure is defined, but you need to fill in the details

In [28]:
class Net(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        #self.fc1 = nn.Linear(input_size, 1024)
        #self.fc2 = nn.Linear(1024, 512)
        #self.fc3 = nn.Linear(512, 256)
        #self.fc4 = nn.Linear(256, 1)
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.out = nn.Linear(256, 1, bias=True)

    def forward(self, x):
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        #x = torch.sigmoid(self.fc4(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = self.out(x)
        x = torch.sigmoid(self.out(x))
        return x

In [29]:
TRAIN_TRIPLETS = '/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/train_triplets.txt'

# load the training data
X, y = get_data(TRAIN_TRIPLETS)
# Create data loaders for the training data
train_loader = create_loader_from_np(X, y, train = True, batch_size=64)
# delete the loaded training data to save memory, as the data loader copies
#del X
#del y

In [30]:
TEST_TRIPLETS = '/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/test_triplets.txt'

# repeat for testing data
X_test, y_test = get_data(TEST_TRIPLETS, train=False)
test_loader = create_loader_from_np(X_test, train = False, batch_size=2048, shuffle=False)
#del X_test
#del y_test

In [31]:
def evaluate_model(model, loader):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = F.binary_cross_entropy(outputs.squeeze(), labels.float(), reduction='sum')
            total_loss += loss.item()
            predicted = (outputs >= 0.5).int()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = total_loss / total
    acc = correct / total
    return loss, acc

In [32]:
def train_model(train_loader, val_loader):
    model = Net(3 * 512)
    model.train()
    model.to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    n_epochs = 5

    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {epoch_loss:.4f}")

        # Validate the model
        val_loss, val_acc = evaluate_model(model, val_loader)
        print(f"Epoch {epoch+1}/{n_epochs}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    return model

In [33]:
kf = KFold(n_splits=2, shuffle=True, random_state=42)

val_losses = []
val_accuracies = []

for fold, (train_index, val_index) in enumerate(kf.split(X)):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    train_loader = create_loader_from_np(X_train, y_train, train=True, batch_size=64)
    val_loader = create_loader_from_np(X_val, y_val, train=False, batch_size=64, shuffle=False)

    model = train_model(train_loader, val_loader)

    val_loss, val_acc = evaluate_model(model, val_loader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Fold {fold+1}: Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

print(f"Average Validation Loss: {np.mean(val_losses):.4f}, Average Validation Accuracy: {np.mean(val_accuracies):.4f}")

model.eval()
predictions = []
# Iterate over the test data
with torch.no_grad(): # We don't need to compute gradients for testing
    for [x_batch] in test_loader:
        x_batch= x_batch.to(device)
        predicted = model(x_batch)
        predicted = predicted.cpu().numpy()
        # Rounding the predictions to 0 or 1
        predicted[predicted >= 0.5] = 1
        predicted[predicted < 0.5] = 0
        predictions.append(predicted)
    predictions = np.vstack(predictions)
np.savetxt("/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/results_task3_4.txt", predictions, fmt='%i')

Epoch 1/5, Train Loss: 0.6658
Epoch 1/5, Val Loss: 0.6086, Val Acc: 32.0665
Epoch 2/5, Train Loss: 0.5938
Epoch 2/5, Val Loss: 0.5886, Val Acc: 32.0852
Epoch 3/5, Train Loss: 0.5718
Epoch 3/5, Val Loss: 0.5838, Val Acc: 32.0762
Epoch 4/5, Train Loss: 0.5581
Epoch 4/5, Val Loss: 0.5767, Val Acc: 32.0894
Epoch 5/5, Train Loss: 0.5442
Epoch 5/5, Val Loss: 0.5649, Val Acc: 32.0947
Fold 1: Validation Loss: 0.5649, Validation Accuracy: 32.0947
Epoch 1/5, Train Loss: 0.6672
Epoch 1/5, Val Loss: 0.6188, Val Acc: 32.0644
Epoch 2/5, Train Loss: 0.5940
Epoch 2/5, Val Loss: 0.5850, Val Acc: 32.0678
Epoch 3/5, Train Loss: 0.5711
Epoch 3/5, Val Loss: 0.5831, Val Acc: 32.0889
Epoch 4/5, Train Loss: 0.5538
Epoch 4/5, Val Loss: 0.5668, Val Acc: 32.0950
Epoch 5/5, Train Loss: 0.5409
Epoch 5/5, Val Loss: 0.5628, Val Acc: 32.0987
Fold 2: Validation Loss: 0.5628, Validation Accuracy: 32.0987
Average Validation Loss: 0.5638, Average Validation Accuracy: 32.0967
